# Assignment 10

<font size = "5">
Print your name below

In [1]:
print('Joy Qu')

Joy Qu


<font size = "5">

Import pandas package

In [2]:
# Import the pandas library
import pandas as pd

<font size = "5">

Import SQL packages

In [3]:
# Write your answer here:
# Import create_engine from sqlalchemy
from sqlalchemy import create_engine

# Import URL from sqlalchemy.engine
from sqlalchemy.engine import URL

# Import text from sqlalchemy
from sqlalchemy import text

<font size = "5">

Establish a connection to the SQL server and database ```postgres``` <br>
from Python as done in  class  using <br>
```URL()``` and ```create_engine()```<br>
As we are are in the default database, you can leave out the database argument.

Note: You may have to change some of the login details <br>
depending on your computer

In [5]:
# Write your answer here:

url_server = URL.create(
    "postgresql",
    host = 'localhost',
    username = 'postgres',
    port = 5432,
    password  = '')
connection = create_engine(url_server)


ModuleNotFoundError: No module named 'psycopg2'


<font size = "5">

(a) Upload dataset to SQL

<font size = "3">

- Read the files ```wdi_2018-2023.xlsx``` in the ```data_raw``` folder. <br>
This is a file containing data from the world development indicators for <br>
2018-2023.
- Upload the data to your local SQL server using the <br>
"to_sql" command. Call the new dataset ```wdi_data```
 in the server

In [ ]:
# Write your own code






<font size = "5">

(b) Subset and sort

<font size = "3">

For the following command use ```pd.read_sql()```

- Use ```WHERE``` to get a subset of "wdi_data" in the SQL server to  <br>
which only selects entries related to the USA, <br>
using the column "country_code"
- Chain this with ```ORDER BY year``` to sort in <br>
ascending order <br>

Hints:
- Remember to include a  ```\```  to be able to define strings over multiple lines
- We can include single quotations in the WHERE command <br>
without any additional escape characters

In [ ]:
# Write your answer here:





<font size = "5">

(c) Groupby + Aggregate + Sort

<font size = "3">

For the following command use ```pd.read_sql()```

- From ```wdi_data``` obtain the average of "prop_urban" <br>
by country_name using ```GROUP BY```
- Use ```HAVING ``` to filter countries with an <br>
average proportion of urban population greater than 40%
- Make sure to separate pieces of your logic over seperate lines  by a ```\```

HINT: Use ```HAVING AVG(prop_urban)``` rather than an alias <br>
for the code to work correctly.


In [ ]:
# Write your answer here:




<font size = "5">

(d) Groupby + Aggregate

<font size = "3">

(1) Run a query using ```pd.read_sql()```
- From ```wdi_data```, obtain the average, minimum, and maximum <br>
of "prop_urban" by "year" using ```GROUP BY```
- Use ```ORDER BY ``` to sort by the average of "avg_prop_urban"
- Make sure to separate pieces of your logic over seperate lines  by a ```\```
- Store the output as a pandas object called ```wdi_grouped```

(2) Upload
- Reupload the result to the server as ```wdi_grouped_data``` using ```to_sql```

In [ ]:
# Write your own code




<font size = "5">

(e) Merge

<font size = "3">

This question depends on your answer to question (d).

For the following command use ```pd.read_sql()```

- From ```wdi_data```, ```LEFT JOIN``` the dataset ```wdi_grouped_data``` <br>
using ```year``` as a merging variable. 
- Hint: See Lecture on SQL for how to merge datasets


In [ ]:
# Write your own code


